In [1]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7909679347775186095, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4438115407113985597
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14702781733503161163
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14900894574416560833
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [92]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

In [5]:
#Importing Keras and other useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from glob import glob
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [19]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['Test', 'Training', 'LICENSE', 'readme.md', 'test-multiple_fruits', 'papers']

In [25]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [28]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [27]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Cantaloupe 2        492
Banana Red          490
Mandarine           490
Plum 2              420
Dates               490
Eggplant            468
Grapefruit White    492
Cantaloupe 1        492
Passion Fruit       490
Apple Granny Smith  492
Orange              479
Apple Golden 2      492
Tangelo             490
Potato Red          450
Tomato 4            479
Pepper Green        444
Granadilla          490
Quince              490
Strawberry          492
Onion Red Peeled    445
Cherry Rainier      738
Pineapple           490
Apricot             492
Hazelnut            464
Kiwi                466
Clementine          490
Onion White         438
Cherry Wax Red      492
Grape Pink          492
Apple Red 2         492
Plum 3              900
Melon Piel de Sapo  738
Cherry Wax Yellow   492
Apple Red 3         429
Pear Forelle        702
Banana Lady Finger  450
Grape White 2       490
Pepino              490
Chestnut            450
Grape White 3 

In [29]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [0]:
#2 Approach: Inception v3 pre-trained
from keras_applications.vgg16 import VGG16
from keras_applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import regularizers
from keras.layers.convolutional import *
from keras.models import Model

In [0]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
test_datagenerator = ImageDataGenerator(rescale= 1./255)

In [35]:
#Creating Batches
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=(156, 156), color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=(156, 156), color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', seed = 20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=(156, 156), color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [0]:
inception_v3_model = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(156,156,3))
#Using pretrained
for layer in inception_v3_model.layers:
  layer.trainable=False
#Adding Layers at the top
top = inception_v3_model.output
top = GlobalAveragePooling2D(data_format='channels_last', name='avg_pool')(top)

#Adding Classification layer with softmax activation function(needed for categorical predictions)
predictor = Dense(120 ,activation='softmax', name='predictor')(top)
inception_v3_model = Model(inputs=inception_v3_model.input, outputs=predictor)

In [55]:
inception_v3_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 156, 156, 3)  0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 77, 77, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_377 (BatchN (None, 77, 77, 32)   96          conv2d_377[0][0]                 
__________________________________________________________________________________________________
activation_377 (Activation)     (None, 77, 77, 32)   0           batch_normalization_377[0][0]    
____________________________________________________________________________________________

In [0]:
#Compiling the model
inception_v3_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
#Training and Validating the model: Round 1
wait = 1
epochs = 15
model_name='Inception v3'
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=wait)

In [58]:
trainingMdl = inception_v3_model.fit_generator(train_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping])

Epoch 1/5
1514/1514 [==============================] - 448s 296ms/step - loss: 1.0777 - accuracy: 0.7755 - val_loss: 13.2418 - val_accuracy: 0.1766
Epoch 2/5
1514/1514 [==============================] - 425s 281ms/step - loss: 0.3746 - accuracy: 0.9044 - val_loss: 4.9319 - val_accuracy: 0.1961
Epoch 3/5
1514/1514 [==============================] - 421s 278ms/step - loss: 0.2939 - accuracy: 0.9201 - val_loss: 4.7999 - val_accuracy: 0.2160
Epoch 4/5
1514/1514 [==============================] - 415s 274ms/step - loss: 0.2421 - accuracy: 0.9306 - val_loss: 6.5929 - val_accuracy: 0.2179


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Progetto Advanced Machine Learning/')

In [94]:
from utils import plot_history
history_fig = plot_history(trainingMdl)
history_fig.show()

ModuleNotFoundError: ignored